In [1]:
import requests
import re
import pandas as pd
from xpinyin import Pinyin
from multiprocessing import Pool
from bs4 import BeautifulSoup


In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36'
}

df = pd.DataFrame(columns=[
    '省', '市', '日期', '星期', '最高温度', '最低温度', '天气', '风向风力'
])

In [3]:
def query(province, city, year, city_pinyin, urls):
    global df
    for url in urls:
        print(url)
        html = requests.get(url, headers=headers)
        html.encoding = html.apparent_encoding
        bs = BeautifulSoup(html.text, 'html.parser')
        data = bs.find_all(class_='thrui')
        date = re.compile('class="th200">(.*?)</')
        tem = re.compile('class="th140">(.*?)</')
        time = re.findall(date, str(data))
        temp = re.findall(tem, str(data))
        # print(time)
        # print(temp)
        data = []
        for i in range(len(time)):
            l1 = [province, city, time[i][:10], time[i][10:], temp[i *
                                                                   4 + 0], temp[i * 4 + 1], temp[i * 4 + 2], temp[i*4+3]]
            data.append(l1)
        df = df.append(pd.DataFrame(data, columns=[
                       '省', '市', '日期', '星期', '最高温度', '最低温度', '天气', '风向风力']), ignore_index=True)

In [11]:
citys = pd.read_csv('../VisualWork/city.csv')
# print(citys)

# 获取所有城市的拼音
p = Pinyin()
city_pinyin_dict = {}
for row in citys.itertuples():
    city_pinyin_dict[row[3]] = p.get_pinyin(row[3]).replace('-', '')

# 生成所有的url
url_dict = {}
for i in range(2011, 2023):
    for row in citys.itertuples():
        province = row[2]
        if province != '辽宁':
            continue
        city = row[3]
        city_pinyin = city_pinyin_dict[city]
        link = []
        for j in range(1, 13):
            if j < 10:
                url = 'https://lishi.tianqi.com/{}/{}0{}.html'.format(
                    city_pinyin, i, j)
            else:
                url = 'https://lishi.tianqi.com/{}/{}{}.html'.format(
                    city_pinyin, i, j)
            link.append(url)
        print((province, city, i))
        url_dict[(province, city, i)] = link

('辽宁', '鞍山', 2011)
('辽宁', '台安', 2011)
('辽宁', '岫岩', 2011)
('辽宁', '铁东', 2011)
('辽宁', '鞍山铁西', 2011)
('辽宁', '立山', 2011)
('辽宁', '千山', 2011)
('辽宁', '本溪', 2011)
('辽宁', '明山', 2011)
('辽宁', '桓仁', 2011)
('辽宁', '平山', 2011)
('辽宁', '溪湖区', 2011)
('辽宁', '南芬', 2011)
('辽宁', '本溪满族', 2011)
('辽宁', '朝阳', 2011)
('辽宁', '建平', 2011)
('辽宁', '双塔', 2011)
('辽宁', '龙城', 2011)
('辽宁', '朝阳县', 2011)
('辽宁', '喀喇沁左翼', 2011)
('辽宁', '大连', 2011)
('辽宁', '金州', 2011)
('辽宁', '普兰店', 2011)
('辽宁', '旅顺', 2011)
('辽宁', '长海', 2011)
('辽宁', '中山', 2011)
('辽宁', '西岗', 2011)
('辽宁', '沙河口', 2011)
('辽宁', '甘井子', 2011)
('辽宁', '旅顺口', 2011)
('辽宁', '丹东', 2011)
('辽宁', '宽甸', 2011)
('辽宁', '元宝', 2011)
('辽宁', '振兴', 2011)
('辽宁', '振安', 2011)
('辽宁', '抚顺', 2011)
('辽宁', '新宾', 2011)
('辽宁', '清原', 2011)
('辽宁', '抚顺县', 2011)
('辽宁', '新抚', 2011)
('辽宁', '望花', 2011)
('辽宁', '顺城', 2011)
('辽宁', '阜新', 2011)
('辽宁', '彰武', 2011)
('辽宁', '海州', 2011)
('辽宁', '新邱', 2011)
('辽宁', '太平', 2011)
('辽宁', '清河门', 2011)
('辽宁', '细河', 2011)
('辽宁', '东洲', 2011)
('辽宁', '阜新蒙古族', 2011)
('辽宁', '葫芦岛',

['https://lishi.tianqi.com/mianyang/202201.html', 'https://lishi.tianqi.com/mianyang/202202.html', 'https://lishi.tianqi.com/mianyang/202203.html', 'https://lishi.tianqi.com/mianyang/202204.html', 'https://lishi.tianqi.com/mianyang/202205.html', 'https://lishi.tianqi.com/mianyang/202206.html', 'https://lishi.tianqi.com/mianyang/202207.html', 'https://lishi.tianqi.com/mianyang/202208.html', 'https://lishi.tianqi.com/mianyang/202209.html', 'https://lishi.tianqi.com/mianyang/202210.html', 'https://lishi.tianqi.com/mianyang/202211.html', 'https://lishi.tianqi.com/mianyang/202212.html']


In [5]:
# query('四川', '绵阳', '2019', 'mianyang', url_dict[('四川', '绵阳', 2022)])
# df.to_csv('../VisualWork/Total_weather.csv', index=False)
# 两行测试代码
# 使用多进程并发调用query函数
def run(args):
    province, city, year = args
    query(province, city, year,
          city_pinyin_dict[city], url_dict[(province, city, year)])


In [12]:
if __name__ == '__main__':
    pool = Pool()
    pool.map(run, url_dict.keys())
    pool.close()
    pool.join()

In [9]:
print(df)

Empty DataFrame
Columns: [省, 市, 日期, 星期, 最高温度, 最低温度, 天气, 风向风力]
Index: []
